In [32]:
# pip install urllib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
You should consider upgrading via the 'C:\Users\1\AppData\Local\Programs\Python\Python36-32\python.exe -m pip install --upgrade pip' command.


In [2]:
import SQLServer_settings as s
import pandas as pd
from sqlalchemy import create_engine,MetaData
from pandas.io import sql
import pyodbc
import urllib


cnxn = pyodbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    f'Server={s.server};'
    f'Database=benchmark;'
    f'UID={s.username};'
    f'PWD={s.password};')


conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+s.server+';DATABASE=benchmark;UID='+s.username+';PWD='+ s.password)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn))



In [3]:
# Download and read csv file
df = pd.read_csv('https://globaldatalab.org/assets/2019/09/SHDI%20Complete%203.0.csv',
                 encoding='ISO-8859–1', dtype='str')
# df = pd.read_csv("SHDI_data.csv")
df = df.reset_index()
# Rename that new column 'id'
# Cosmos DB needs one column named 'id'.
df = df.rename(columns={'index': 'id'})
# Convert the id column to a string - this is a document database.
df['id'] = df['id'].astype(str)
result = df

In [4]:
try:
    result.to_sql("myTable", con=engine, schema='dbo', if_exists='append', index=False)
except Exception as e:
    print(e)
    print('Error when trying to export to DB')
    raise e

In [ ]:
df2 = pd.read_sql_query("SELECT * FROM myTable;", con=engine)